In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
from konlpy.tag import Mecab
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from ngboost import NGBRegressor
from catboost import CatBoostRegressor, Pool

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   일자              1205 non-null   datetime64[ns]
 1   요일              1205 non-null   int64         
 2   본사정원수           1205 non-null   int64         
 3   본사휴가자수          1205 non-null   int64         
 4   본사출장자수          1205 non-null   int64         
 5   본사시간외근무명령서승인건수  1205 non-null   int64         
 6   현본사소속재택근무자수     1205 non-null   float64       
 7   조식메뉴            1205 non-null   object        
 8   중식메뉴            1205 non-null   object        
 9   석식메뉴            1205 non-null   object        
 10  중식계             1205 non-null   float64       
 11  석식계             1205 non-null   float64       
 12  month           1205 non-null   int64         
 13  corona          1205 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(7), object(3)
me

In [4]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [5]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [6]:
# 띄어쓰기 및 오타 수정

train.at[1142, '중식메뉴'] = '쌀밥/곤드레밥/찰현미밥 된장찌개 돼지고추장불고기 버섯잡채 삼색물만두무침 겉절이김치/양념장 견과류샐러드*요거트D'

train['중식메뉴'] = train['중식메뉴'].str.replace('삽겹', '삼겹')

In [7]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['요일'] = train['일자'].dt.day_name().str[:2].map({'Mo' : 5, 'Tu' : 4, 'We' : 3, 'Th' : 2, 'Fr' : 1})
test['요일'] = test['일자'].dt.day_name().str[:2].map({'Mo' : 5, 'Tu' : 4, 'We' : 3, 'Th' : 2, 'Fr' : 1})

train['month'] = train['일자'].dt.month
test['month'] = test['일자'].dt.month

# 재택근무자가 존재할 경우 코로나 사태임을 가정
train['corona'] = [1 if x > 0 else 0 for x in train['현본사소속재택근무자수']]
test['corona'] = [1 if x > 0 else 0 for x in test['현본사소속재택근무자수']]

In [10]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona
1200,2021-01-20,3,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0,1,1
1201,2021-01-21,2,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0,1,1
1202,2021-01-22,1,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0,1,1
1203,2021-01-25,5,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0,1,1
1204,2021-01-26,4,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,쌀밥/귀리밥/찰현미밥 들깨미역국 교촌간장치킨 옥수수콘치즈구이 가지고추장무침 포기김치...,(New)할라피뇨멸치주먹밥 잔치국수 수제고기육전 쑥갓나물 양파초절임 깍두기,1015.0,480.0,1,1


In [9]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,month,corona
0,2021-01-27,3,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,1,1
1,2021-01-28,2,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,1,1
2,2021-01-29,1,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치,1,1
3,2021-02-01,5,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치,2,1
4,2021-02-02,4,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치,2,1


In [12]:
# 다음 출근날까지의 일수 차이

train['shift_1day'] = train.일자.shift(-1)
test['shift_1day'] = test.일자.shift(-1)

# 마지막 데이터에 대한 처리
train.at[1204, 'shift_1day'] = datetime(2021,1,27)
test.at[49, 'shift_1day'] = datetime(2021,4,12)

train['day_gap'] = (train.shift_1day - train.일자).astype(str)
test['day_gap'] = (test.shift_1day - test.일자).astype(str)

train.at[0, 'day_gap'] = '1 days'
test.at[0, 'day_gap'] = '1 days'

In [14]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
1200,2021-01-20,3,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0,1,1,2021-01-21,1 days 00:00:00.000000000
1201,2021-01-21,2,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0,1,1,2021-01-22,1 days 00:00:00.000000000
1202,2021-01-22,1,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0,1,1,2021-01-25,3 days 00:00:00.000000000
1203,2021-01-25,5,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0,1,1,2021-01-26,1 days 00:00:00.000000000
1204,2021-01-26,4,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,쌀밥/귀리밥/찰현미밥 들깨미역국 교촌간장치킨 옥수수콘치즈구이 가지고추장무침 포기김치...,(New)할라피뇨멸치주먹밥 잔치국수 수제고기육전 쑥갓나물 양파초절임 깍두기,1015.0,480.0,1,1,2021-01-27,1 days 00:00:00.000000000


In [15]:
def get_holiday_score(x) :
    
    s = int(re.sub(r'[^0-9]', '', x))
    if s == 1 :
        return 0
    else :
        return 1

In [16]:
train['day_gap'] = train.day_gap.apply(get_holiday_score)
test['day_gap'] = test.day_gap.apply(get_holiday_score)

In [17]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap
1200,2021-01-20,3,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0,1,1,2021-01-21,1
1201,2021-01-21,2,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0,1,1,2021-01-22,1
1202,2021-01-22,1,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0,1,1,2021-01-25,1
1203,2021-01-25,5,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0,1,1,2021-01-26,1
1204,2021-01-26,4,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,쌀밥/귀리밥/찰현미밥 들깨미역국 교촌간장치킨 옥수수콘치즈구이 가지고추장무침 포기김치...,(New)할라피뇨멸치주먹밥 잔치국수 수제고기육전 쑥갓나물 양파초절임 깍두기,1015.0,480.0,1,1,2021-01-27,1


In [18]:
# 이건 어떻게 단정...?

train['야근요일'] = train.요일.apply(lambda x : 1 if (x == 1) or (x == 3) else 0)
test['야근요일'] = test.요일.apply(lambda x : 1 if (x == 1) or (x == 3) else 0)

In [19]:
train['중식메뉴'] = train['중식메뉴'].str.split(' ')
train['석식메뉴'] = train['석식메뉴'].str.split(' ')

test['중식메뉴'] = test['중식메뉴'].str.split(' ')
test['석식메뉴'] = test['석식메뉴'].str.split(' ')

In [21]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,month,corona,shift_1day,day_gap,야근요일
1200,2021-01-20,3,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,"[쌀밥/흑미밥/찰현미밥, 아욱국, 수제함박스테이크, 견과류마카로니범벅, 생깻잎지, ...","[김치볶음밥, 미니쫄우동*맛살튀김, 브로콜리깨소스무침, 계란후라이, 고들빼기무침, ...",1093.0,421.0,1,1,2021-01-21,1,1
1201,2021-01-21,2,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,"[쌀밥/수수밥/찰현미밥, 냉이된장국, 동파육, 봄동전, 청경채/버섯숙회*초장, 무생...","[흑미밥, 쇠고기무국, 삼치양념구이, 비엔나채소볶음, 숙주나물당근무침, 포기김치, ]",832.0,353.0,1,1,2021-01-22,1,0
1202,2021-01-22,1,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,"[전주비빔밥*약고추장, 계란파국, 요거닭, 올방개묵무침, 파프리카해초무침, 포기김치...","[흑미밥, 수제비국, 수제맛쵸킹탕수육, 유부채소겨자냉채, 참나물무침, 갓김치/겉절이...",579.0,217.0,1,1,2021-01-25,1,1
1203,2021-01-25,5,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,"[쌀밥/흑미밥/찰현미밥, 전주식콩나물해장국, 돈육간장불고기, 깐풍연근, 연두부*달래...","[흑미밥, 열무된장국, 장어강정*데리야끼소스, 깻잎쌈*생강채, 오이선, 포기김치, ]",1145.0,502.0,1,1,2021-01-26,1,0
1204,2021-01-26,4,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,"[쌀밥/귀리밥/찰현미밥, 들깨미역국, 교촌간장치킨, 옥수수콘치즈구이, 가지고추장무침...","[(New)할라피뇨멸치주먹밥, 잔치국수, 수제고기육전, 쑥갓나물, 양파초절임, 깍두...",1015.0,480.0,1,1,2021-01-27,1,0


In [22]:
def get_token(data) :
    tokens = []
    for token in data :
        s_list = []
        for t in token :
            if t.startswith('(N') :
                s_list.append(t)
            elif (t.startswith('(') == False) & (len(t) > 1) :
                s_list.append(t)
            else :
                pass
        tokens.append(s_list)
    return tokens

In [23]:
# 개수를 좀 더 정확하게 세기 위함 인듯

train['중식_토큰'] = get_token(train['중식메뉴'])
train['석식_토큰'] = get_token(train['석식메뉴'])

In [24]:
test['중식_토큰'] = get_token(test['중식메뉴'])
test['석식_토큰'] = get_token(test['석식메뉴'])

In [25]:
train['중식메뉴수'] = train.중식_토큰.apply(len)
train['석식메뉴수'] = train.석식_토큰.apply(len)

test['중식메뉴수'] = test.중식_토큰.apply(len)
test['석식메뉴수'] = test.석식_토큰.apply(len)

In [26]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식계,month,corona,shift_1day,day_gap,야근요일,중식_토큰,석식_토큰,중식메뉴수,석식메뉴수
1200,2021-01-20,3,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,"[쌀밥/흑미밥/찰현미밥, 아욱국, 수제함박스테이크, 견과류마카로니범벅, 생깻잎지, ...","[김치볶음밥, 미니쫄우동*맛살튀김, 브로콜리깨소스무침, 계란후라이, 고들빼기무침, ...",...,421.0,1,1,2021-01-21,1,1,"[쌀밥/흑미밥/찰현미밥, 아욱국, 수제함박스테이크, 견과류마카로니범벅, 생깻잎지, ...","[김치볶음밥, 미니쫄우동*맛살튀김, 브로콜리깨소스무침, 계란후라이, 고들빼기무침, ...",7,6
1201,2021-01-21,2,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,"[쌀밥/수수밥/찰현미밥, 냉이된장국, 동파육, 봄동전, 청경채/버섯숙회*초장, 무생...","[흑미밥, 쇠고기무국, 삼치양념구이, 비엔나채소볶음, 숙주나물당근무침, 포기김치, ]",...,353.0,1,1,2021-01-22,1,0,"[쌀밥/수수밥/찰현미밥, 냉이된장국, 동파육, 봄동전, 청경채/버섯숙회*초장, 무생...","[흑미밥, 쇠고기무국, 삼치양념구이, 비엔나채소볶음, 숙주나물당근무침, 포기김치]",7,6
1202,2021-01-22,1,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,"[전주비빔밥*약고추장, 계란파국, 요거닭, 올방개묵무침, 파프리카해초무침, 포기김치...","[흑미밥, 수제비국, 수제맛쵸킹탕수육, 유부채소겨자냉채, 참나물무침, 갓김치/겉절이...",...,217.0,1,1,2021-01-25,1,1,"[전주비빔밥*약고추장, 계란파국, 요거닭, 올방개묵무침, 파프리카해초무침, 포기김치...","[흑미밥, 수제비국, 수제맛쵸킹탕수육, 유부채소겨자냉채, 참나물무침, 갓김치/겉절이김치]",7,6
1203,2021-01-25,5,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,"[쌀밥/흑미밥/찰현미밥, 전주식콩나물해장국, 돈육간장불고기, 깐풍연근, 연두부*달래...","[흑미밥, 열무된장국, 장어강정*데리야끼소스, 깻잎쌈*생강채, 오이선, 포기김치, ]",...,502.0,1,1,2021-01-26,1,0,"[쌀밥/흑미밥/찰현미밥, 전주식콩나물해장국, 돈육간장불고기, 깐풍연근, 연두부*달래...","[흑미밥, 열무된장국, 장어강정*데리야끼소스, 깻잎쌈*생강채, 오이선, 포기김치]",7,6
1204,2021-01-26,4,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,"[쌀밥/귀리밥/찰현미밥, 들깨미역국, 교촌간장치킨, 옥수수콘치즈구이, 가지고추장무침...","[(New)할라피뇨멸치주먹밥, 잔치국수, 수제고기육전, 쑥갓나물, 양파초절임, 깍두...",...,480.0,1,1,2021-01-27,1,0,"[쌀밥/귀리밥/찰현미밥, 들깨미역국, 교촌간장치킨, 옥수수콘치즈구이, 가지고추장무침...","[(New)할라피뇨멸치주먹밥, 잔치국수, 수제고기육전, 쑥갓나물, 양파초절임, 깍두기]",7,6


In [27]:
# 식재료
# 중식에만 처리함 -> 메뉴가 중식에서 더 중요할 것으로 판단했기 때문...?

def get_ingredient(data) :
    
    ing_df = pd.DataFrame(np.zeros((data.shape[0], 7)), columns = ['해산물', '소', '돼지', '닭', '오리', '채소', '재료_기타'])

    for t in range(data.shape[0]) :
        token = data.중식_토큰.str[2][t]
        if '연어' in token or'골뱅이' in token or'열기' in token or'조기' in token or'탕수어' in token or'양장피' in token or'홍어' in token or'명태' in token or'적어' in token or'장어' in token or'동태' in token or'산슬' in token or'코다리' in token or'가자미' in token or'해물' in token or'생선' in token or'새우' in token or'꽁치' in token or'갈치' in token or'임연수' in token or'삼치' in token or'고등어' in token or'굴비' in token or'오징어' in token or'쭈꾸미' in token or'주꾸미' in token or'낙지' in token or'문어' in token :
            ing_df.at[t, '해산물'] = 1
        elif '왕갈비' in token or'소갈비' in token or'장조림' in token or'불고기' in token or'차돌' in token or'육전' in token or'너비아니' in token or'떡갈비' in token or(token.startswith('소') & (token.startswith('소세') == False)) or '함박' in token or'쇠고기' in token or'소고기' in token or'쇠' in token :
            ing_df.at[t, '소'] = 1
        elif '궁보계정' in token or'삼계탕' in token or'윙' in token or'유린기' in token or'깐풍'in token or'닭' in token or'치킨' in token or'후라이드' in token :
            ing_df.at[t, '돼지'] = 1
        elif '폭립' in token or'오향장육' in token or'동파육' in token or'히레카츠' in token or'순대' in token or'미트볼' in token or'등갈비' in token or'소세지' in token or'목살' in token or'탕수육' in token or'제육' in token or'돈' in token or'돼지' in token or'두루치기' in token or'삼겹' in token or'보쌈' in token or'족발' in token :
            ing_df.at[t, '닭'] = 1
        elif '오리' in token :
            ing_df.at[t, '오리'] = 1
        elif token.endswith('두부') or '꼬치산적' in token or '고추' in token or'양파' in token or'부추' in token or'고구마' in token or'감자' in token or'깻잎' in token or'샐러드' in token or'시금치' in token or'야채' in token :
            ing_df.at[t, '채소'] = 1
        else :
            ing_df.at[t, '재료_기타'] = 1
            
    return ing_df

In [28]:
train = pd.concat([train, get_ingredient(train)], axis = 1)
test = pd.concat([test, get_ingredient(test)], axis = 1)

In [29]:
def get_recipe(data, col) :
    tm = col[:2]
    cat = ['전', '무침','튀김', '찜', '볶음', '조림', '구이', '훈제', '조리_기타']
    recipe_df = pd.DataFrame(np.zeros((data.shape[0], 9)), columns = [f'{tm}_{x}' for x in cat])

    for t in range(data.shape[0]) :
        try :
            token = data[col][t]
            if '고추잡채' in token or '궁보계정' in token or '산슬' in token or token.endswith('잡채') or '마파두부' in token or '두루치기' in token or '닭갈비' in token or token.endswith('볶음') or '볶음' in token :
                recipe_df.at[t, f'{tm}_볶음'] = 1 
            elif token.endswith('데리야끼') or token.endswith('립') or '함박' in token or '그라탕' in token or token.endswith('갈비') or '주물럭' in token or '스테이크' in token or token.endswith('구이') or '불고기' in token or '구이' in token :
                recipe_df.at[t, f'{tm}_구이'] = 1
            elif '전병' in token or token.endswith('전') :
                recipe_df.at[t, f'{tm}_전'] = 1
            elif token.endswith('김치말이') or token.endswith('만두') or '보쌈' in token or '수육' in token or token.endswith('찜') or '찜' in token :
                recipe_df.at[t, f'{tm}_찜'] = 1
            elif '파채' in token or token.endswith('무침') or token.endswith('샐러드') or '양장피' in token :
                recipe_df.at[t, f'{tm}_무침'] = 1
            elif '오향장육' in token or '동파육' in token or token.endswith('조림') :
                recipe_df.at[t, f'{tm}_조림'] = 1
            elif '통닭' in token or token.endswith('새우') or '강정' in token or '미트볼' in token or '프리타타' in token or '카츠' in token or '깐풍' in token or '고로케' in token or '유린기' in token or '탕수' in token or token.endswith('닭') or token.endswith('치킨') or token.endswith('튀김') or '너겟' in token or token.endswith('강정') or '가스' in token or '까스' in token or '핑거' in token or '텐더' in token or '커틀렛' in token or '커틀릿' in token :
                recipe_df.at[t, f'{tm}_튀김'] = 1
            elif '훈제' in token :
                recipe_df.at[t, f'{tm}_훈제'] = 1
            else :
                recipe_df.at[t, f'{tm}_조리_기타'] = 1
        except :
            recipe_df.at[t, f'{tm}_조리_기타'] = 1
    return recipe_df

In [30]:
train['중식_메인요리'] = train.중식_토큰.str[2]
test['중식_메인요리'] = test.중식_토큰.str[2]

In [31]:
train['석식_메인요리'] = train.석식_토큰.str[2]
test['석식_메인요리'] = test.석식_토큰.str[2]

In [32]:
train = pd.concat([train, get_recipe(train, '중식_메인요리')], axis = 1)
test = pd.concat([test, get_recipe(test, '중식_메인요리')], axis = 1)

In [33]:
train = pd.concat([train, get_recipe(train, '석식_메인요리')], axis = 1)
test = pd.concat([test, get_recipe(test, '석식_메인요리')], axis = 1)

In [34]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,중식_조리_기타,석식_전,석식_무침,석식_튀김,석식_찜,석식_볶음,석식_조림,석식_구이,석식_훈제,석식_조리_기타
0,2016-02-01,5,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 오징어찌개, , 쇠불고기, (쇠고기:호주산...","[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 육개장, , 자반고등어구이, , 두부조림,...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2016-02-02,4,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 김치찌개, , 가자미튀김, , 모둠소세지구...","[콩나물밥*양념장, (쌀,현미흑미:국내산), 어묵국, , 유산슬, (쇠고기:호주산)...",...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2016-02-03,3,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"[카레덮밥, (쌀,현미흑미:국내산), 팽이장국, , 치킨핑거, (닭고기:국내산), ...","[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 청국장찌개, , 황태양념구이, (황태:러시...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2016-02-04,2,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 쇠고기무국, , 주꾸미볶음, , 부추전, ...","[미니김밥*겨자장, (쌀,현미흑미:국내산), 우동, , 멕시칸샐러드, , 군고구마,...",...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-02-05,1,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 떡국, , 돈육씨앗강정, (돼지고기:국내산...","[쌀밥/잡곡밥, (쌀,현미흑미:국내산), 차돌박이찌개, (쇠고기:호주산), 닭갈비,...",...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

In [36]:
test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

In [37]:
X1 = train[['요일', '야근요일', '출근', 'day_gap', '휴가비율', '출장비율', '야근비율', 'month',  '중식메뉴수', '해산물', '소', '돼지', '닭', '오리', '채소', '재료_기타', '중식_전', '중식_무침', '중식_튀김', '중식_찜', 
       '중식_볶음', '중식_조림', '중식_구이', '중식_훈제', '중식_조리_기타']]

In [38]:
target1 = test[X1.columns]

In [39]:
X2 = train[['corona', '석식메뉴수', 'month', '야근요일', 'day_gap', '요일', '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '석식_전', '석식_무침', 
       '석식_튀김', '석식_찜', '석식_볶음', '석식_조림', '석식_구이', '석식_훈제', '석식_조리_기타']]

In [40]:
target2 = test[X2.columns]

In [41]:
y1 = train.중식계
y2 = train.석식계

In [42]:
kf = KFold(n_splits = 15, random_state = 718, shuffle = True)

In [43]:
# catboostregressor

cb = CatBoostRegressor(iterations = 20000, learning_rate = 0.01, depth = 4, eval_metric = 'MAE', silent = True, loss_function = 'MAE')

In [ ]:
cb_pred_1 = np.zeros((target1.shape[0]))
mae_list = []
for tr_idx, val_idx in kf.split(X1):
    tr_x, val_x = X1.iloc[tr_idx], X1.iloc[val_idx]
    tr_y, val_y = y1.iloc[tr_idx], y1.iloc[val_idx]
    train_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 2000, use_best_model = True, verbose = 5000)
    best = cb.best_iteration_
    pred = cb.predict(val_x, ntree_end = best)
    mae = mean_absolute_error(val_y, pred)
    mae_list.append(mae)
    print(f'FOLD MAE = {mae}')
    sub_pred = cb.predict(target1, ntree_end = best) / 15
    cb_pred_1 += sub_pred
print(f'\n{cb.__class__.__name__} MAE = {np.mean(mae_list)}')

0:	learn: 165.7463513	test: 174.5120978	best: 174.5120978 (0)	total: 197ms	remaining: 1h 5m 38s
5000:	learn: 44.9400972	test: 62.4313854	best: 62.4159233 (4919)	total: 15.3s	remaining: 45.8s
10000:	learn: 38.7687488	test: 61.6856636	best: 61.6107954 (9618)	total: 30.9s	remaining: 30.9s
Stopped by overfitting detector  (2000 iterations wait)

bestTest = 61.61079536
bestIteration = 9618

Shrink model to first 9619 iterations.
FOLD MAE = 61.61079720393186
0:	learn: 166.7817161	test: 162.5860484	best: 162.5860484 (0)	total: 3.77ms	remaining: 1m 15s
5000:	learn: 44.6408447	test: 75.3369110	best: 75.1939521 (4078)	total: 14.1s	remaining: 42.4s
Stopped by overfitting detector  (2000 iterations wait)

bestTest = 75.19395214
bestIteration = 4078

Shrink model to first 4079 iterations.
FOLD MAE = 75.19444696951089
0:	learn: 167.5657819	test: 150.2906163	best: 150.2906163 (0)	total: 5.27ms	remaining: 1m 45s
5000:	learn: 44.4978086	test: 75.5052743	best: 75.4986569 (4988)	total: 14.3s	remaining: 4